In [3]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from skimage import io, color, filters
from ipywidgets import interact, FloatSlider, Dropdown



ROOT = Path('/Users/mn/Desktop/g_images/imgs_to_update/') # path to greyscale
imgs_all = list(ROOT.glob('*.jpg'))

img_path = imgs_all[1]
img_name = str(img_path).split('.')[0].split('/')[-1]

image = io.imread(ROOT / img_path, as_gray=True)

def binarize(blur_sigma, threshold_method, threshold_delta):
    blurred = filters.gaussian(image, sigma=blur_sigma)
    # apply thresholding
    if threshold_method == 'Otsu':
        thresh = filters.threshold_otsu(image)
    elif threshold_method == 'Li':
        thresh = filters.threshold_li(image)
    elif threshold_method == 'Isodata':
        thresh = filters.threshold_isodata(image)
    elif threshold_method == 'Mean':
        thresh = filters.threshold_mean(image)
    elif threshold_method == 'Minimum':
        thresh = filters.threshold_minimum(image)
    
    binary = blurred > (thresh + threshold_delta)

    plt.imshow(binary, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.show()

# sliders
threshold_slider = FloatSlider(min=-1, max=1, step=0.001, value=0, description='Threshold')
blur_sigma_slider = FloatSlider(min=0, max=9, step=0.1, value=1.0, description='Blur Sigma')
threshold_method_widget = Dropdown(options=['Otsu', 'Li', 'Isodata', 'Mean', 'Minimum'],
                                   description='Threshold Method')


interact(binarize, threshold_method=threshold_method_widget, blur_sigma=blur_sigma_slider, threshold_delta=threshold_slider);

interactive(children=(FloatSlider(value=1.0, description='Blur Sigma', max=9.0), Dropdown(description='Thresho…

In [5]:
# get slider values
threshold_method = threshold_method_widget.value
threshold_delta = threshold_slider.value
blur_sigma = blur_sigma_slider.value

# save mooney

mooney_name = img_name.strip("dis_")


gray_image = io.imread(img_path, as_gray=True)

blurred = filters.gaussian(gray_image, sigma=blur_sigma)
# apply thresholding
if threshold_method == 'Otsu':
    thresh = filters.threshold_otsu(gray_image)
elif threshold_method == 'Li':
    thresh = filters.threshold_li(gray_image)
elif threshold_method == 'Isodata':
    thresh = filters.threshold_isodata(gray_image)
elif threshold_method == 'Mean':
    thresh = filters.threshold_mean(gray_image)
elif threshold_method == 'Minimum':
    thresh = filters.threshold_minimum(gray_image)

binary = blurred > (thresh + threshold_delta)

plt.imsave((ROOT / f"{mooney_name}.jpg"), binary, cmap='gray')
# print(ROOT / f"{mooney_name}.jpg")